# Definitions

In [111]:
# parameter.py
from itertools import chain, tee, accumulate
from copy import copy, deepcopy
from functools import lru_cache, wraps
from numpy import nan

def cached_property(func):
    maxsize = 1
    return property(lru_cache(maxsize)(func))

class Parameter:
    '''
    Parameter class used in Model class.
    
    Once its instance is created, its property is cacked.
    '''
    def __init__(self,name,*sub_params,abstract=False):
        if not isinstance(name,str):
            raise TypeError("{} is not string".format(name))
        self._name = name
        self.sub_params = []
        
        self._abstract = abstract
        
        if sub_params:
            self.add(*sub_params)
    
    def create_cache(self):
        _ = self.composite
        _ = len(self)
        _ = self.flattened
    
    
    @cached_property
    def abstract(self):
        return self._abstract
    
    @cached_property
    def composite(self):
        return bool(self.sub_params)
    
    #def __len__(self):
    #    if not self.composite:
    #        return 1
    #    else:
    #        return sum([len(sub_param) for sub_param in self.sub_params])    
    
    @cached_property
    def __len__(self):
        '''
        The number of dimension of 'self'.
        '''
        return len(self.flattened)
    
    def __getitem__(self,index):
        if index in self.sub_param_names:
            return 
    
    def add(self,first,*sub_params):  # "first" is required if sub_params is a single parameter
        '''
        compose argument parameters into self.sub_params.
        '''
        _sub_params = chain((first,),sub_params)
        self.sub_params.extend(_sub_params)
        #self._len = len(list(self.flattened))  # calculate statistics when it updated
        self.create_cache()
    
    #def __or__(self,param):
    #    self.sub_params.extend(param.sub_params)
    #    self._name = "+".join([self._name,param.name])
    
    @property
    #@lru_cache()
    def _flattened(self):
        '''
        If self is a not-composite parameter, return '[self,]' .
        Else it returns a iterator of all '.sub_params' in 'self', recursively.
        
        Note: The return is cached to acceralete computing.
        Note: Users cannot use the rerurn directly, while they can do through `flattened`
        '''
        if not self.composite:
            return [self,] 
        else:
            return chain.from_iterable([p._flattened for p in self.sub_params])  # Note: This is relatively time consunming.
    
    @cached_property
    def flattened(self):
        '''
        Note: lru_cache()  does not works as attempted 
        because the hashed value of the argument 'self' is not changed by 
        assigning different values.
        NO!!! IT WORKS BECAUSE IT RETURNS A LIST OBJECT!!!
        '''
        return list(self._flattened)
    
    @property
    def name(self):
        return self._name
    
    @property
    def name_quoted(self):
        return "'{}'".format(self._name)
    
    @property
    def sub_param_names(self):
        return [p.name for p in self.sub_params]
    
    @property
    def sub_param_names_flattened(self):
        return [p.name for p in self._flattened]
    
    @property
    def sub_param_names_quoted(self):
        return ["'{}'".format(sub_param.name) for sub_param in self.sub_params]
    
    @property
    def lens_sub_params(self):
        return [len(sub_params) for sub_params in self.sub_params]
    
    @property
    def n_sub_params(self):
        return len(self.sub_param_names)
    
    @property
    def value_flattened(self):
        #'''
        #Note: Actual value is stored in the top level Parameter for quick accessing as "self._value".
        #The variable "self._value" is bounded with those of sub_params, so if it is changed,
        #those of sub_params are also chainged.
        #'''  # <- THIS FEATURE IS OVERED
        return [p._value for p in self._flattened]
        #if not self.value_assigned:
        #    raise TypeError("No value is assigned!")
        #else:
        #    return self._value
        
    @property
    def value(self):
        if not self.composite:
            return self._value
        else:
            return [p.value for p in self.sub_params]
    
    @property
    def value_assigned(self):
        #return (self._value is not None)
        return hasattr(self,"_value")
    
    def _assign_value(self,value):
        self._value = value
    
    def split_values(self,values):
        if len(values) != len(self):
            errmes = "argument {} has a different shape from {}: ({})"
            errmes = errmes.format(values,self.name_quoted,self.sub_param_names_flattened)
            raise ValueError(errmes)
        start,stop = tee(accumulate(self.lens_sub_params))
        start = chain([0,],start)
        flatten = lambda x: x[0] if len(x)==1 else x 
        sub_values = [flatten(values[i:j]) for i,j in zip(start,stop)]
        
        return sub_values
    
    def assign_value(self,first,*values):
        '''
        assign a specific value given by the arguments.
        
        e.g.
        
        self.assgin_value(1.0)
        
        self.assign_value(1.0, 2.0, 3.0)
        '''
        if self.abstract:
            raise AttributeError("{} is abstract parameter, not assignable".format(self.name))
        
        value = list(chain([first,],values))
        n = len(self)
            
        if len(value) != n:
            err_mes = self.name + ": "
            err_mes += "The length of value {} (={}) is inconsistent to the length of {} (={})"
            raise TypeError(err_mes.format(value,len(value),self._name,n))
        else:
            #### IT IS TIME CONSUMING. ####
            #if self.composite:
            #    start,stop = tee(accumulate(self.lens_sub_params))
            #    start = chain([0,],start)
            #    sub_values = [value[i:j] for i,j in zip(start,stop)]
            #    #print([(p,val) for p,val in zip(self.sub_params,sub_values)])
            #    [p.assign_value(*val) for p,val in zip(self.sub_params,sub_values)]
            
            #if self.composite:
            #    start,stop = tee(accumulate(self.lens_sub_params))
            #    start = chain([0,],start)
            #    sub_values = [value[i:j] for i,j in zip(start,stop)]
            #    #print([(p,val) for p,val in zip(self.sub_params,sub_values)])
            #    [p.assign_value(*val) for p,val in zip(self.sub_params,sub_values)]
            #    
            #else:
            #    self._value = first if not isinstance(first,list) else first[0]
            
            [p._assign_value(val) for p,val in zip(self._flattened,value)]
            
                #print(first,isinstance(first,list),type(first))
            
            
    #def clear_values(self):
    #    del self._value
        
    @property
    def _str_base(self):
        '''
        create string.
        '''
        #_type = "Single" if not self.composite else "Composite"
        
        output_list = []
        
        if self.abstract: output_list.append("Abstract")
        output_list.append("Parameter:")
        output_list.append(self.name_quoted)
        
        if self.composite:
            sub_param_names = ", ".join(self.sub_param_names_quoted)
            output_list.append("[{} sub_params: {} ]".format(self.n_sub_params,sub_param_names))
        elif self.value_assigned:
            output_list.append("( value: {} )".format(self._value))
        
        _str = "< " + " ".join(output_list) + " >"
        _str_subs = chain.from_iterable([["\t"+s for s in sub_param._str_base] for sub_param in self.sub_params])
        #print([_repr,*_repr_subs])
        return [_str,*_str_subs]
    
    def __str__(self):
        return "\n".join(self._str_base)
    
    def __repr__(self): 
        return self._str_base[0]

In [112]:
# model.py
# from .parameter import Parameter
import functools

#def _classmethod_param_packed(func,cls_Model,prefix_original="_",prefix_converted=""):
#    '''
#    decorate raw 'func' (func(x,a,b,..., [aa,bb,..], [aaa,bbb,...], ...)) 
#    to take flatten arguments (func(x,y,a,b,...,aa,bb,...,aaa,bbb,...)) .
#    The function before the decoration are saved as a classmethod of func.__class__.
#    decorated function are saved as an attribute attempted by users.
#    
#    !!! IT DOES NOT WORK NAIVELY BECAUSE THE DECORATOR IS CALLED BEFORE THE DEFINOTION OF CLS_MODEL !!!
#    !!! IT MUST BE CALLED BY CLASS !!!
#    '''
#    # See https://qiita.com/chankane/items/3909e9f2d1c5910cc60b
#    def decorator(func):
#        arg_list = func.__code__.co_varnames[:func.__code__.co_argcount]
#        params_names = cls_Model.parameter.sub_param_names
#    
#        @functools.wraps(func)
#        def wrapped_func(cls,*args, p=None,**kwargs):
#            '''
#            wrapped function.
#        
#            args: original arguments of func, except for parameter. 
#            kwargs: flatten parameter list.
#            '''
#        
#            if p is None:
#                errmes = "keyword argument 'p' is not specified or None.\n"
#                errmes += "(Automatically wrapped function must take parameters like func(x,y,p=[...]) )"
#                raise TypeError(mes)
#                
#            splited_values = parameter.split_values(p)
#            dict_params = {p_name:val for p_name,val in zip(params_names,splited_values) if (p_name in arg_list)}
#            kwargs.update(dict_params)
#            return func(self,*args,**kwargs)
#        
#        wrapped_func.__name__ = prefix_converted + func.__name__
#        setattr(cls_Model,prefix_original+func.__name__,classmethod(func))
#        
#        return classmethod(wrapped_func)
#    
#    return decorator


class _ModelMeta(type):
    function_prefix = "func_"

    def __new__(cls,cls_name,cls_bases,cls_dict):
        '''
        Create an class attrtibute `sub_models`.
        Create an attribute 'parameter' as an abstract parameter.
        Create `flat` functions as attributes from functions like 'func_'.
        '''
    ## extend `param_names` to include all names of the `sub_models`.
    #    param_names = cls_dict["param_names"]
    #    if not cls_dict["sub_models"]:
    #        param_names.extend([cls_sub_model.parameter for cls_sub_model in cls_dict["sub_models"]])  
        if not ("sub_models" in cls_dict) : cls_dict["sub_models"] = []
            
        parameters = [Parameter(name,abstract=True) for name in cls_dict["param_names"]]
        
        for cls_sub_model in cls_dict["sub_models"]:
            parameters.append(cls_sub_model.parameter)
            cls_dict[cls_sub_model.__name__] = cls_sub_model
            #print(parameters)
            
        cls_dict["parameter"] = Parameter("Parameter_"+cls_name,*parameters,abstract=True)
        
        # define 'flat' function
        for attr in list(cls_dict):
            len_prefix = len(_ModelMeta.function_prefix)
            if attr[:len_prefix] == _ModelMeta.function_prefix:
                wrapped_func = _ModelMeta.function_wrapper(cls_dict[attr],cls_dict["parameter"])
                cls_dict[attr[len_prefix:]] = classmethod(wrapped_func)
                cls_dict[attr] = classmethod(cls_dict[attr])
                #print(cls_dict)
                
        #def classmethod_param_packed(func,cls_Model,prefix_original="_",prefix_converted=""):
        #    return _classmethod_param_packed(func,cls,prefix_original,prefix_converted)
        #
        #cls_dict["classmethod_param_packed"] = classmethod(classmethod_param_packed)
        
        return super().__new__(cls,cls_name,cls_bases,cls_dict)
    

    @staticmethod
    def function_wrapper(func,parameter):
        '''
        convert 'raw' function into 'flat' function:
        
        'raw': f(x,y,a,b,Parameter_Model1,Parameter_Model2)
            where 'Parameter_X' is flatened.
            
        'flat': f(x,y,p)
            where 'p' is flattened.
        '''
        # See https://qiita.com/chankane/items/3909e9f2d1c5910cc60b
        
        arg_list = func.__code__.co_varnames[:func.__code__.co_argcount]
        param_list = parameter.sub_param_names
        
        @functools.wraps(func)
        def wrapped_func(self,*args, p=None,**kwargs):
            '''
            wrapped function.
            
            args: original arguments of func, except for parameter. 
            kwargs: flatten parameter list.
            '''
            if p is None:
                raise TypeError("keyword argument 'p' is not specified or None.\n(Automatically wrapped function must take parameters like func(x,y,p=[...]) )")
            splited_params = parameter.split_values(p)
            dict_params = {name:val for name,val in zip(parameter.sub_param_names,splited_params) if (name in arg_list)}
            #print(kwargs,dict_params)
            kwargs.update(dict_params)
            return func(self,*args,**kwargs)
        
        return wrapped_func
    

    
class Model(metaclass=_ModelMeta):
    param_names = []
    sub_models = []
    
#    def __new__(cls):
#        '''
#        Define the 'param_names' of an instance.
#        Generate attibutes of parameters automatically. 
#        '''
#        
#        cls_name = cls.__name__
#        cls_bases = cls.__bases__
#        cls_dict = cls.__dict__
#        
#        parameters = [Parameter(name) for name in cls_dict["param_names"]]
#        
#        if "sub_models" in cls_dict:
#            cls.sub_models = []
#           for cls_sub_model in cls_dict["sub_models"]:
#                sub_model = cls_sub_model()
#                cls.sub_models.append(sub_model)
#                parameters.append(sub_model.parameter)
#                cls_dict[cls_sub_model.__name__] = sub_model
#           #print(parameters)
#            
#        cls_dict["parameter"] = Parameter("Parameter_"+cls_name,*parameters)
#        return super().__new__(cls,cls_name,cls_bases,cls_dict)
    
    def __init__(self):
        '''
        create Model instances of submodels as well as Parameter instance.
        '''
        parameters = [Parameter(name) for name in self.param_names]  # Note: It masks 'Model.parameter'.
        sub_models = []
        
        for cls_sub_model in self.sub_models:
            sub_model = cls_sub_model()
            sub_models.append(sub_model)
            parameters.append(sub_model.parameter)
            setattr(self, cls_sub_model.__name__, sub_model)
        
        #print(parameters)
            
        self.parameter = Parameter("Parameter_"+self.__class__.__name__,*parameters)
        self.sub_models = sub_models
        
        # initialize parameter
        [p.assign_value(nan) for p in self.parameter.flattened]
        
    def assign_value(self,*p):
        self.parameter.assign_value(*p)
    
    def split_params(self,values):
        return self.parameter.split_values(values)




# Test

In [113]:
# test_parameter.py
import unittest

class TestParaemter(unittest.TestCase):
    '''
    test class of the class 'Parameter'.
    '''

    def setUp(self):
        self.a = Parameter("a")
        self.b = Parameter("b")
        self.g = Parameter("g")
        self.dm = Parameter("DM param",self.a,self.b,self.g)
        self.re = Parameter("re")
        self.odds = Parameter("odds")
        self.stellar = Parameter("stellar param",self.re,self.odds)
        self.anib = Parameter("anib")
        self.dsph = Parameter("dSph param", self.anib, self.stellar, self.dm)
    
    def tearDown(self):
        del self
    
    
    def test_print(self):
        string = "< Parameter: 'dSph param' [3 sub_params: 'anib', 'stellar param', 'DM param' ] >\n"
        string += "\t< Parameter: 'anib' >\n"
        string += "\t< Parameter: 'stellar param' [2 sub_params: 're', 'odds' ] >\n"
        string += "\t\t< Parameter: 're' >\n"
        string += "\t\t< Parameter: 'odds' >\n"
        string += "\t< Parameter: 'DM param' [3 sub_params: 'a', 'b', 'g' ] >\n"
        string += "\t\t< Parameter: 'a' >\n"
        string += "\t\t< Parameter: 'b' >\n"
        string += "\t\t< Parameter: 'g' >"
		
        output = str(self.dsph)
        
        self.assertEqual(string,output,"string unmatched\n{}\n{}".format(string,output))
    
    
    def test_assign_value(self):
        value1 = [0,1.,2.,3.,4.,5.]
        value1_ = [0,[1,2],[3,4,5]]
        value2 = [5.,6.,7.,8.,9.,10.]
        self.dsph.assign_value(*value1)
        self.assertEqual(value1,self.dsph.value_flattened)
        self.assertEqual(value1_,self.dsph.value)
        
        # check whether the parameter can update its value properly
        self.dsph.assign_value(*value2)
        self.assertEqual(value2,self.dsph.value_flattened)
    
    

In [114]:
# test_model.py
# import unittest
class TestModel(unittest.TestCase):
    '''
    test class of the class 'Parameter'.
    '''

    def setUp(self):
        class Model1(Model):
            param_names = ["a","b","c"]

            def func_f_model1(cls,x,a,b,c):
                return {"f_model1":{"x":x,"a":a,"b":b,"c":c}}
        
            
        class Model2(Model):
            param_names = ["d","e","f"]

            def func_f_model2(cls,x,d,e,f):
                return {"f_model2":{"x":x,"d":d,"e":e,"f":f}}
        
        class Model3(Model):
            param_names = ["A","B"]
            sub_models = [Model1,Model2]
            
            def func_f_model3(cls,x,A,B,Parameter_Model1,Parameter_Model2):
                ret = {"f_model3":{"x":x,"A":A,"B":B,"Parameter_Model1":Parameter_Model1,"Parameter_Model2":Parameter_Model2}}
                ret.update(cls.Model1.f_model1(x,p=Parameter_Model1))
                ret.update(cls.Model2.f_model2(x,p=Parameter_Model2))
                return ret
        
        self.Model1 = Model1
        self.Model2 = Model2
        self.Model3 = Model3
            

    def tearDown(self):
        del self
    
    def test_parameter(self):
        string = "< Abstract Parameter: 'Parameter_Model3' [4 sub_params: 'A', 'B', 'Parameter_Model1', 'Parameter_Model2' ] >\n"
        string += "\t< Abstract Parameter: 'A' >\n"
        string += "\t< Abstract Parameter: 'B' >\n"
        string += "\t< Abstract Parameter: 'Parameter_Model1' [3 sub_params: 'a', 'b', 'c' ] >\n"
        string += "\t\t< Abstract Parameter: 'a' >\n"
        string += "\t\t< Abstract Parameter: 'b' >\n"
        string += "\t\t< Abstract Parameter: 'c' >\n"
        string += "\t< Abstract Parameter: 'Parameter_Model2' [3 sub_params: 'd', 'e', 'f' ] >\n"
        string += "\t\t< Abstract Parameter: 'd' >\n"
        string += "\t\t< Abstract Parameter: 'e' >\n"
        string += "\t\t< Abstract Parameter: 'f' >"
		
        
        output = str(self.Model3.parameter)
        self.assertEqual(string,output,msg="string unmatched:\n{}\n{}".format(string,output))
        
    def test_assign_value(self):
        value1 = [0,1,2,3,4,5,6,7]
        value1_ = [0,1,[2,3,4],[5,6,7]]
        value2 = [1,2,3,4,5,6,7,8]
        
        with self.assertRaises(AttributeError):
            self.Model3.assign_value(value1)
        
        model = self.Model3()
        model.assign_value(*value1)
        self.assertEqual(value1,model.parameter.value_flattened)
        self.assertEqual(value1_,model.parameter.value)
        
        # check whether the parameter can update its value properly
        model.assign_value(*value2)
        self.assertEqual(value2,model.parameter.value_flattened)
    
    def test_model_func(self):
        
        #self.assertIsInstance(self.Model1.f_model1,classmethod)
        #self.assertIsInstance(self.Model2.f_model2,classmethod)
        #self.assertIsInstance(self.Model3.f_model3,classmethod)
        
        #self.assertIsInstance(self.Model1.func_f_model1,classmethod)
        #self.assertIsInstance(self.Model2.func_f_model2,classmethod)
        #self.assertIsInstance(self.Model3.func_f_model3,classmethod)
        
        
        x,A,B,a,b,c,d,e,f = list(range(9))
        ret_Model1 = {"f_model1":dict(x=x,a=a,b=b,c=c)}
        ret_Model2 = {"f_model2":dict(x=x,d=d,e=e,f=f)}
        ret_Model3 = {"f_model3":dict(x=x,A=A,B=B,Parameter_Model1=[a,b,c],Parameter_Model2=[d,e,f])}
        ret_Model3.update(ret_Model1)
        ret_Model3.update(ret_Model2)
        
        self.assertEqual(self.Model1.func_f_model1(x,a,b,c),ret_Model1)
        self.assertEqual(self.Model1.f_model1(x,p=[a,b,c]),ret_Model1)
        self.assertEqual(self.Model2.f_model2(x,p=[d,e,f]),ret_Model2)
        self.assertEqual(self.Model3.func_f_model3(x,A,B,[a,b,c],[d,e,f]),ret_Model3)
        self.assertEqual(self.Model3.f_model3(x,p=[A,B,a,b,c,d,e,f]),ret_Model3)
        
        model1,model2,model3 = self.Model1(), self.Model2(), self.Model3()
        self.assertEqual(model1.func_f_model1(x,a,b,c),ret_Model1)
        self.assertEqual(model1.f_model1(x,p=[a,b,c]),ret_Model1)
        self.assertEqual(model2.f_model2(x,p=[d,e,f]),ret_Model2)
        self.assertEqual(model3.func_f_model3(x,A,B,[a,b,c],[d,e,f]),ret_Model3)
        self.assertEqual(model3.f_model3(x,p=[A,B,a,b,c,d,e,f]),ret_Model3)
        
        with self.assertRaises(ValueError):
            model1.f_model1(x,p=[a,b,c,d])
            model1.f_model1(x,p=[a])
            model3.f_model3(x,p=[a,b])
            model3.f_model3(x,p=[A,A,A,B,a,b,c,d,e,f])

In [107]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.003s

OK


In [70]:
{"f_model1":dict(x=1,A=2,B=3)}

{'f_model1': {'x': 1, 'A': 2, 'B': 3}}

# Profiling

In [116]:
%load_ext line_profiler
%lprun -f Model.assign_value Model.assign_value(*[1.,2.,3.,4.,5.])
#%timeit dsph.assign_value(*[1.,2.,3.,4.,5.])
#print(dsph)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


AttributeError: 'float' object has no attribute 'parameter'

In [30]:
%timeit dsph.value

2.6 µs ± 32.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [22]:
mylist = [1,2,3,4,5]
data = [1,2,3,4,5]
%timeit for i in range(len(mylist)): mylist[i] = data[i]

380 ns ± 0.401 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [23]:
%%timeit
for i in range(len(mylist)):
    mylist[i]

274 ns ± 0.983 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [71]:
model.assign_value(*[1,2,3,4,5,6,7,8])
print(model.parameter)
print(model.parameter.value)

< Parameter: 'Parameter_Model12' [4 sub_params: 'A', 'B', 'Parameter_Model1', 'Parameter_Model2' ] >
	< Parameter: 'A' ( value: 1 ) >
	< Parameter: 'B' ( value: 2 ) >
	< Parameter: 'Parameter_Model1' [3 sub_params: 'a', 'b', 'c' ] >
		< Parameter: 'a' ( value: 3 ) >
		< Parameter: 'b' ( value: 4 ) >
		< Parameter: 'c' ( value: 5 ) >
	< Parameter: 'Parameter_Model2' [3 sub_params: 'd', 'e', 'f' ] >
		< Parameter: 'd' ( value: 6 ) >
		< Parameter: 'e' ( value: 7 ) >
		< Parameter: 'f' ( value: 8 ) >
[1, 2, [3, 4, 5], [6, 7, 8]]


In [72]:
model.assign_value(*[1,2,3,4,5,6,7,8])
print(model.parameter.value)

[1, 2, [3, 4, 5], [6, 7, 8]]


In [113]:
%timeit list(model.split_params(list(range(8))))
list(model.split_params(list(range(8))))

7.85 µs ± 22.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


[0, 1, [2, 3, 4], [5, 6, 7]]

In [106]:
%timeit model.split_params_2(list(range(8)))
model.split_params_2(list(range(8)))

7.81 µs ± 29.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


[0, 1, [2, 3, 4], [5, 6, 7]]

In [83]:
list(zip(*[iter(range(16))]*3))

[(0, 1, 2), (3, 4, 5), (6, 7, 8), (9, 10, 11), (12, 13, 14)]

In [85]:
[*[iter(range(16))]*3]

In [115]:
dir(model.func_test)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [119]:
model.func_test.__code__

<code object func_test at 0x7fc887d16ed0, file "<ipython-input-111-1b4850fc654d>", line 96>

In [147]:
import functools
import inspect

# a.x -> type(a).__dict__['x'].__get__(a, type(a))
# A.x -> A.__dict__['x'].__get__(None, A)
#
# --> __get__の第一引数がNoneのときは、クラスから呼ばれた時。
# そこで、A.__dict__['x']に__get__を持ったオブジェクトを置いとくと、
# いい感じにHackできる。だたし、__get__(obe,objtype)の返り値にも注意。
# a.x の場合は、返り値はメソッドである必要がある。つまり、関数の第一引数を固定したやつ。
# A.x の場合は、気にしないでよし（エラーにする）。

#def pure_instancemethod(func):
#    functools.wraps(func)
#    def wrapped_func(self,*args,**kwargs):
#        if not isinstance(self,)
#            raise TypeError("クラスメソッドじゃないよ")

class PureInstanceMethod:
    def __init__(self,func):
        self.func = func
    
    def __get__(self,obj,objtype):
        if obj is None:  # クラスからの呼び出し：
            raise TypeError("クラスから呼ぶなタコ")
            
        else: # インスタンスからの呼び出し
            def instancemethod(*args,**kwargs):
                return self.func(obj,*args,**kwargs)
            return instancemethod # メソッドを返してあげよう
        
    def __call__(self,*args,**kwargs):
        return self.func(*args,**kwargs)
            
class MyClass:
    
    @PureInstanceMethod
    def mymethod(self,*x):
        print(x)
    
    # MyClass.__dict__["mymethod"] = pureinstancemethod(mymethod) とほぼ同じ。
    # つまりここにはmymethodで初期化されたpureinstancemethodのインスタンスが入る。

x = list(range(4))
myclass = MyClass()

In [151]:
myclass.mymethod(*[1,2,3])  # [0,1,2,3]

(1, 2, 3)


In [152]:
MyClass.mymethod(*[1,2,3])  # [1,2,3]

TypeError: クラスから呼ぶなタコ

In [127]:
myclass.mymethod.__self__

In [128]:
myclass